# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Installation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 6 - Phenotype prediction

## Working with a GEM model

For this practical session, we will be using the following model:
- _iMM904_ model which contains the whole metabolism of _S. cerevisiae_ -> file: ../data/iMM904.xml

You can read more about _S. cerevisiae_ iMM804 model (Mo et al., 2009) in the following links:
- https://bmcsystbiol.biomedcentral.com/articles/10.1186/1752-0509-3-37
- http://bigg.ucsd.edu/models/iMM904

This exercise consists of exploring the phenotype prediction tools of COBRApy. Thus, the following steps will be followed:
- Perform an pFBA simulation using an aerobic/anaerobic conditions;
- Perform an FVA simulation;
- Perform gene deletions;
- Perform single deletions
- Perform production envelopes

In [1]:
# imports
import cobra
import escher

In [2]:
# Loading a model
model_path = '../data/iMM904.xml'
model = cobra.io.read_sbml_model(model_path)

model

Name,iMM904
Memory address,219681a4f08
Number of metabolites,1226
Number of reactions,1577
Number of genes,905
Number of groups,0
Objective expression,1.0*BIOMASS_SC5_notrace - 1.0*BIOMASS_SC5_notrace_reverse_93090
Compartments,"cytosol, extracellular space, mitochondria, peroxisome/glyoxysome, endoplasmic reticulum, vacuole, golgi apparatus, nucleus"


## Phenotype Prediction

### pFBA - Aerobiosis

In [3]:
#check model exchanges
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

Episterol ester  yeast specific exchange -> (0.0, 999999.0)
Episterol exchange -> (0.0, 999999.0)
Ergosterol exchange -> (0.0, 999999.0)
Ergosterol ester  yeast specific exchange -> (0.0, 999999.0)
1 3 beta D Glucan exchange -> (0.0, 999999.0)
Ethanolamine exchange -> (0.0, 999999.0)
2 Hydroxybutyrate exchange -> (0.0, 999999.0)
Ethanol exchange -> (0.0, 999999.0)
Fe2+ exchange -> (-999999.0, 999999.0)
Fecosterol exchange -> (0.0, 999999.0)
2 methylbutyl acetate exchange -> (0.0, 999999.0)
2 methylbutyraldehyde exchange -> (0.0, 999999.0)
2 methyl 1 butanol exchange -> (0.0, 999999.0)
2 methylpropanal exchange -> (0.0, 999999.0)
2 phenylethanol exchange -> (0.0, 999999.0)
Fecosterol ester  yeast specific exchange -> (0.0, 999999.0)
FMN exchange -> (0.0, 999999.0)
3 Carboxy 3 hydroxy 4 methylpentanoate exchange -> (0.0, 999999.0)
3 Methylbutanal exchange -> (0.0, 999999.0)
Formate exchange -> (0.0, 999999.0)
D-Fructose exchange -> (0.0, 999999.0)
3mop exchange -> (0.0, 999999.0)
4-Amino

In [4]:
model.exchanges.EX_o2_e

Reaction identifier,EX_o2_e
Name,O2 exchange
Memory address,0x21925a88348
Stoichiometry,o2_e <=> O2 O2 <=>
GPR,
Lower bound,-2.0
Upper bound,999999.0


In [5]:
#performing pfba simulation
pfba_solution = cobra.flux_analysis.pfba(model)
model.summary(pfba_solution)

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,1.611,0,0.00%
o2_e,EX_o2_e,2,0,0.00%
pi_e,EX_pi_e,0.05691,0,0.00%
so4_e,EX_so4_e,0.02225,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-18.02,1,36.29%
etoh_e,EX_etoh_e,-15.82,2,63.70%
for_e,EX_for_e,-0.001488,1,0.00%
h2o_e,EX_h2o_e,-5.64,0,0.00%


### Setting up anaerobiosis

In [6]:
# setting the environmental conditions to replicate an anaerobic medium
anaerobic_conditions = {'EX_ergstest_SC_e': (-2, 1000),
                        'EX_pro__L_e': (-2, 1000), 
                        'EX_ptd1ino_SC_e':(-2, 1000), 
                        'EX_zymst_e': (-2, 1000),
                        'EX_o2_e': (0, 1000)}

for exchange_id, bds in anaerobic_conditions.items():
    model.exchanges.get_by_id(exchange_id).bounds = bds

### pFBA - Anaerobiosis

In [7]:
#performing pfba simulation
pfba_solution = cobra.flux_analysis.pfba(model)
model.summary(pfba_solution)

Metabolite,Reaction,Flux,C-Number,C-Flux
ergstest_SC_e,EX_ergstest_SC_e,0.0001507,1695,0.42%
glc__D_e,EX_glc__D_e,10,6,98.08%
nh4_e,EX_nh4_e,1.073,0,0.00%
pi_e,EX_pi_e,0.04255,0,0.00%
pro__L_e,EX_pro__L_e,0.1316,5,1.08%
ptd1ino_SC_e,EX_ptd1ino_SC_e,6.027E-05,4140,0.41%
so4_e,EX_so4_e,0.01664,0,0.00%
zymst_e,EX_zymst_e,0.0003229,27,0.01%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-16.79,1,31.42%


### Flux Variability Analysis (FVA)

In [8]:
#performing fva simulation
fva_solution = cobra.flux_analysis.flux_variability_analysis(model, fraction_of_optimum=0.1)
fva_solution

,minimum,maximum
CITtcp,0.000000,0.000000
13BGH,0.000000,8.550000
13BGHe,0.000000,0.000000
13GS,0.024425,8.574425
16GS,0.000000,0.000000
...,...,...
PYDXO,0.000000,0.000000
PYK,2.749686,36.949750
PYNP2r,-8.550000,8.550000
PYR5CDm,0.000000,4.807041


In [9]:
# maximum theoretical production rates of Ethanol (EX_eth_e), Carbon dioxide (EX_co2_e)

print(f'Theoretical production rate interval for Ethanol:', fva_solution.loc['EX_etoh_e', 'minimum'], 'to', fva_solution.loc['EX_etoh_e', 'maximum'], 'mmol/gDW/h')
print(f'Theoretical production rate interval for Carbon Dioxide:', fva_solution.loc['EX_co2_e', 'minimum'], 'to', fva_solution.loc['EX_co2_e', 'maximum'], 'mmol/gDW/h')

Theoretical production rate interval for Ethanol: 0.0 to 21.092216193102132 mmol/gDW/h
Theoretical production rate interval for Carbon Dioxide: 0.0 to 23.163365984183777 mmol/gDW/h


### Simulating Deletions

#### Knock outs

In [10]:
model.genes.YPL262W

Gene identifier,YPL262W
Name,FUM1
Memory address,0x219257e3f88
Functional,True
In 2 reaction(s),"FUM, FUMm"


In [11]:
model.reactions.FUM

Reaction identifier,FUM
Name,Fumarase
Memory address,0x21926382548
Stoichiometry,fum_c + h2o_c <=> mal__L_c Fumarate + H2O H2O <=> L-Malate
GPR,YPL262W
Lower bound,-999999.0
Upper bound,999999.0


In [12]:
model.reactions.FUMm

Reaction identifier,FUMm
Name,Fumarase mitochondrial
Memory address,0x219263be548
Stoichiometry,fum_m + h2o_m <=> mal__L_m Fumarate + H2O H2O <=> L-Malate
GPR,YPL262W
Lower bound,-999999.0
Upper bound,999999.0


In [13]:
#knock out the YPL262W (FUM1) gene associated with FUM and FUMm reactions
with model:
    model.genes.YPL262W.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('FUM1 mutant growth rate: ', pfba_solution.fluxes['BIOMASS_SC5_notrace'])
    print('Ethanol production rate: ', pfba_solution.fluxes['EX_etoh_e'])

FUM1 mutant growth rate:  0.21347511876332417
Ethanol production rate:  17.62964664175147


In [14]:
builder = escher.Builder(map_name='iMM904.Central carbon metabolism', model=model, reaction_data=pfba_solution.fluxes)
builder

Builder(reaction_data={'CITtcp': 0.0, '13BGH': 0.0, '13BGHe': 0.0, '13GS': 0.24225156477262028, '16GS': 0.0, '…

#### Single Deletions

In [15]:
#single reaction deletion
double_reaction_deletion_results = cobra.flux_analysis.single_reaction_deletion(model)
double_reaction_deletion_results

,ids,growth,status
0,{ATPS3m},0.21524,optimal
1,{EX_taur_e},0.21524,optimal
2,{BTSr},0.21524,optimal
3,{FBA},0.21524,optimal
4,{TAGL_SC},0.21524,optimal
...,...,...,...
1572,{EX_thmpp_e},0.21524,optimal
1573,{HACD5p},0.21524,optimal
1574,{EX_spmd_e},0.21524,optimal
1575,{ORPT},0.00000,optimal


In [16]:
#single gene deletion
single_gene_deletion_results = cobra.flux_analysis.single_gene_deletion(model)
single_gene_deletion_results

,ids,growth,status
0,{YHL032C},0.21524,optimal
1,{YJL214W},0.21524,optimal
2,{YPL091W},0.21524,optimal
3,{YGR192C},0.21524,optimal
4,{YML004C},0.21524,optimal
...,...,...,...
900,{YHL016C},0.21524,optimal
901,{YIR028W},0.21524,optimal
902,{YIL160C},0.21524,optimal
903,{YHL011C},0.21524,optimal
